In [1]:
import os, json
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [2]:
from langsmith import Client
client = Client()

In [3]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['GPT4_MODEL_NAME']
embedding_model = os.environ['EMBEDDING_MODEL_NAME']
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=embedding_model, chunk_size=1)
vector_store_endpoint: str = os.environ['AZURE_COGNITIVE_SEARCH_ENDPOINT']
vector_store_password: str = os.environ['AZURE_COGNITIVE_SEARCH_KEY']
index_name: str = "contoso-coffee-index"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_endpoint,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [4]:
# Perform a vector search
docs = vector_store.vector_search(
    # query = "What are your famous bakery items?", 
    query = "Can I place an order for 1 Mocha, 1 lemon cake, 1 blueberry smoothie, and 1 Chicken sandwich?",  
    k=10,
)

for doc in docs:
    print(doc.page_content)

 
        Item Name: Chicken Sandwich
        Chicken Sandwich details:
            Price: 5.5$
            Category: Sandwiches
            Description: Delicious sandwich filled with chicken and veggies.
    
 
        Item Name: Blueberry Smoothie
        Blueberry Smoothie details:
            Price: 5.5$
            Category: Smoothies
            Description: Delicious smoothie made with fresh blueberries.
    
 
        Item Name: Blueberry Muffin
        Blueberry Muffin details:
            Price: 2.5$
            Category: Bakery
            Description: Soft and moist muffin packed with fresh blueberries and a crumb topping.
    
 
        Item Name: Chocolate Smoothie
        Chocolate Smoothie details:
            Price: 6.0$
            Category: Smoothies
            Description: Rich and creamy smoothie made with chocolate.
    
 
        Item Name: Club Sandwich
        Club Sandwich details:
            Price: 6.0$
            Category: Sandwiches
            Descript

In [5]:
# Perform a hybrid search
docs = vector_store.hybrid_search(
    # query = "What are your famous bakery items?", 
    query = "Can I place an order for 1 Mocha, 1 lemon cake, 1 blueberry smoothie, and 1 Chicken sandwich?",  
    k=15,
)

for doc in docs:
    print(doc.page_content)

 
        Item Name: Chicken Sandwich
        Chicken Sandwich details:
            Price: 5.5$
            Category: Sandwiches
            Description: Delicious sandwich filled with chicken and veggies.
    
 
        Item Name: Club Sandwich
        Club Sandwich details:
            Price: 6.0$
            Category: Sandwiches
            Description: Triple-decker sandwich with chicken, bacon, lettuce, and tomato.
    
 
        Item Name: Chocolate Smoothie
        Chocolate Smoothie details:
            Price: 6.0$
            Category: Smoothies
            Description: Rich and creamy smoothie made with chocolate.
    
 
        Item Name: Lemon Cake
        Lemon Cake details:
            Price: 4.0$
            Category: Bakery
            Description: Sweet and tangy lemon cake with a moist crumb.
    
 
        Item Name: Mocha
        Mocha details:
            Price: 4.5$
            Category: Coffees
            Description: Delicious combination of coffee, milk and ch

In [6]:
# Perform a semantic hybrid search
vector_store.semantic_configuration_name = "contoso-semantic-config"
docs = vector_store.semantic_hybrid_search(
    # query = "What are your famous bakery items?", 
    query = "Can I place an order for 1 Mocha, 1 lemon cake, 1 blueberry smoothie, and 1 Chicken sandwich?",  
    k=15,
)

for doc in docs:
    print(doc.page_content)

 
        Item Name: Lemon Cake
        Lemon Cake details:
            Price: 4.0$
            Category: Bakery
            Description: Sweet and tangy lemon cake with a moist crumb.
    
 
        Item Name: Mocha
        Mocha details:
            Price: 4.5$
            Category: Coffees
            Description: Delicious combination of coffee, milk and chocolate.
    
 
        Item Name: Bagel
        Bagel details:
            Price: 1.5$
            Category: Bakery
            Description: Delicious, dense and chewy bread product, perfect for a quick breakfast.
    
 
        Item Name: Lemon Tea
        Lemon Tea details:
            Price: 2.5$
            Category: Tea or Chai
            Description: A tea with a lemon flavor.
    
 
        Item Name: Carrot Cake
        Carrot Cake details:
            Price: 4.0$
            Category: Bakery
            Description: Moist, sweet cake with a rich carrot and cinnamon flavor.
    
 
        Item Name: Blueberry Muffin
   

In [4]:
# Using Memory object to track chat_history
import time
from datetime import datetime  
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import CosmosDBChatMessageHistory
import random

chat = AzureChatOpenAI(temperature=0.0,
    max_tokens=400,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    deployment_name=os.environ['GPT4_MODEL_NAME'],
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
    streaming=True,  
)

# Create CosmosDB instance from langchain cosmos class.
cosmos = CosmosDBChatMessageHistory(
    cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
    cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
    cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
    connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
    session_id="Agent-Test-Session" + str(random.randint(1, 1000)),
    user_id="Agent-Test-User" + str(random.randint(1, 1000))
    )

# prepare the cosmosdb instance
cosmos.prepare_cosmos()

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 20})

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, input_key="question", output_key="answer", chat_memory=cosmos)


COMBINE_QUESTION_PROMPT_TEMPLATE = """
    As a chatbot for Contoso Coffee Inc., your tasks are:
	• Menu contains coffees, teas, bakery items, sandwiches and smoothies. Guide customers through the menu and answer their inquiries about the menu items. Receive and process orders, and handle complaints with empathy.
	• Since you are representing Contoso, NEVER MENTION 'I am an AI language model'. Do not use 'their', use 'our' instead. And use "please let me know" instead of "please let our staff know". Focus solely on cafe-related queries and ordering. 
	• Keep responses brief and clear. Long responses may disengage the customers. Do not provide additional information, such as description, until explicitly asked for. Do not repeat anything until asked for, especially if you already mentioned in the conversation history.
	• Inform the customer politely if an item they request is not available in the listed menu items below.
	• If a customer likes to update or cancel the order, please help accordingly.
	• Capture any additional notes the customer may have for the menu items.
	• When listing an item in the order, mention the actual price of the item in parenthesis without multiplying it with the quantity.
	• For subtotal of the order, calculate it step-by-step: 
		○ Total price for each ordered item = price of each ordered item * quantity of each ordered item (do this for all ordered items)
		○ Subtotal = sum of total prices for all ordered items
		○ Tax amount = 0.095 * Subtotal
		○ Total = Subtotal + Tax amount
	• Do not provide the total amount of the order until asked for. If and when asked, only mention the final sum in dollars without providing the entire calculation. As mentioned earlier, remember to be brief in your responses.
	• At the end of each conversation, always obtain the customer's name, even if provided in a single word, and attach it to the order.
	• Once the customer provides their name, confirm it and state that the order can be picked up at our cafe in 15 minutes.
	• Contoso accepts only pickup orders. Payment is accepted only at the store, and if a customer suggests paying over the phone, inform them that payments are accepted solely at the store.
Remember to always maintain a polite and professional tone.

Conversaion History:
{chat_history}

Question: 
{question}

Menu Items: 
{context}

Answer:"""

COMBINE_QUESTION_PROMPT = PromptTemplate(
    template=COMBINE_QUESTION_PROMPT_TEMPLATE, input_variables=["chat_history", "question", "context"]
)


qa_chain = ConversationalRetrievalChain.from_llm(
    llm=chat,
    retriever=retriever,
    memory=memory,
    chain_type="stuff",
    verbose=False,
    combine_docs_chain_kwargs={"prompt": COMBINE_QUESTION_PROMPT}
)

In [5]:
def remove_price(input_string: str) -> str:
    import re
    # Use regular expression to remove parentheses and their contents
    output_string = re.sub(r'\s*\([^)]*\)', '', input_string)
    return output_string

In [6]:
import time

print("Welcome to Contoso Coffee Inc! How can I assist you?\n")
while True:
    query = input()
    if query in ["quit", "exit"]:
        break
    start_time = time.time()
    result = qa_chain({"question": query})
    end_time = time.time()
    execution_time = end_time - start_time
    print("Customer: {0}".format(query))
    print("Assistant: {0}".format(result['answer']))
    print(f"\tTime taken to respond: {round(execution_time)} seconds")


Welcome to Contoso Coffee Inc! How can I assist you?

Customer: hello how can you help me?
Assistant: Hello! I'm here to assist you with your order at Contoso Coffee. We have a variety of items on our menu including coffees, teas, bakery items, sandwiches, and smoothies. If you have any questions about our menu items or if you're ready to place an order, please let me know.
	Time taken to respond: 6 seconds
Customer: I am in mood for french. What french items do you have?
Assistant: We have two French items on our menu. The first one is Cafe au Lait, a coffee drink made with strong hot coffee and scalded milk, priced at $3.5. The second one is Pain au Chocolat, a classic French pastry filled with chocolate, priced at $3.5.
	Time taken to respond: 5 seconds
Customer: Can I order 2 each of the first 2 items
Assistant: Of course, you can order two each of Cafe au Lait and Pain au Chocolat. That would be $3.5 (Cafe au Lait) * 2 + $3.5 (Pain au Chocolat) * 2. Do you have any additional note

In [7]:
cosmos.messages

[HumanMessage(content='hello how can you help me?'),
 AIMessage(content="Hello! I'm here to assist you with your order at Contoso Coffee. We have a variety of items on our menu including coffees, teas, bakery items, sandwiches, and smoothies. If you have any questions about our menu items or if you're ready to place an order, please let me know."),
 HumanMessage(content='I am in mood for french. What french items do you have?'),
 AIMessage(content='We have two French items on our menu. The first one is Cafe au Lait, a coffee drink made with strong hot coffee and scalded milk, priced at $3.5. The second one is Pain au Chocolat, a classic French pastry filled with chocolate, priced at $3.5.'),
 HumanMessage(content='Can I order 2 each of the first 2 items'),
 AIMessage(content='Of course, you can order two each of Cafe au Lait and Pain au Chocolat. That would be $3.5 (Cafe au Lait) * 2 + $3.5 (Pain au Chocolat) * 2. Do you have any additional notes for these items?'),
 HumanMessage(conte

In [12]:
print(cosmos.session_id)
print(cosmos.user_id)

Agent-Test-Session660
Agent-Test-User127


In [13]:
print(memory.buffer_as_str)

Human: hello how can you help me?
AI: Hello! I'm here to assist you with your order at Contoso Coffee. We have a variety of items on our menu including coffees, teas, bakery items, sandwiches, and smoothies. If you have any questions about our menu items or if you're ready to place an order, please let me know.
Human: I am in mood for french. What french items do you have?
AI: We have two French items on our menu. The first one is Cafe au Lait, a coffee drink made with strong hot coffee and scalded milk, priced at $3.5. The second one is Pain au Chocolat, a classic French pastry filled with chocolate, priced at $3.5.
Human: Can I order 2 each of the first 2 items
AI: Of course, you can order two each of Cafe au Lait and Pain au Chocolat. That would be $3.5 (Cafe au Lait) * 2 + $3.5 (Pain au Chocolat) * 2. Do you have any additional notes for these items?
Human: Yes. Can you make my coffee very hot please and less sugar too. Thanks!
AI: Absolutely, we will prepare your Cafe au Lait very

In [17]:
def get_order_items(chat_str: str):
    from langchain.chat_models import AzureChatOpenAI
    from langchain.chains import create_extraction_chain
    from langchain.prompts import ChatPromptTemplate

    llm = AzureChatOpenAI(temperature=0.0,
        max_tokens=500,
        openai_api_base=openai.api_base,
        openai_api_version="2023-07-01-preview",
        deployment_name=os.environ['GPT4_MODEL_NAME'],
        openai_api_key=openai.api_key,
        openai_api_type = openai.api_type    
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a world class algorithm for extracting information in structured formats from a restaurant order conversation."),
            ("human", """You will receive a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant. 
            Your task is to extract and save the pertinent details, specifically the ordered food items, quantity ordered, their prices and any additional notes requested for each ordered item included in the order. 
            Make sure to extract only those items that were ordered. Input: {input}"""),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

    # Schema
    schema = {
        "properties": {
            "item": {"type": "string"},
            "quantity": {"type": "integer"},
            "price": {"type": "integer"},
            "item_notes": {"type": "string"},
        },
        "required": ["item_name", "quantity", "price"],
    }

    # Run chain
    chain = create_extraction_chain(schema=schema, llm=llm, prompt=prompt, verbose=False)
    order_result = chain.run(chat_str)
    return order_result

order_result = get_order_items(memory.buffer_as_str)
order_result



[{'item': 'Cafe au Lait',
  'quantity': 2,
  'price': 3.5,
  'item_notes': 'Very hot, less sugar'},
 {'item': 'Pain au Chocolat', 'quantity': 2, 'price': 3.5, 'item_notes': ''},
 {'item': 'Mocha', 'quantity': 1, 'price': 4.5, 'item_notes': ''},
 {'item': 'Lemon Cake', 'quantity': 1, 'price': 4.0, 'item_notes': ''},
 {'item': 'Blueberry Smoothie', 'quantity': 1, 'price': 5.5, 'item_notes': ''},
 {'item': 'Chicken Sandwich', 'quantity': 1, 'price': 5.5, 'item_notes': ''}]

In [23]:
def extract_customer_name(chat_str: str):
    from langchain.chat_models import AzureChatOpenAI
    from langchain.chains import create_extraction_chain
    from langchain.prompts import ChatPromptTemplate

    llm = AzureChatOpenAI(temperature=0.0,
        max_tokens=500,
        openai_api_base=openai.api_base,
        openai_api_version="2023-07-01-preview",
        deployment_name="gpt-4",
        openai_api_key=openai.api_key,
        openai_api_type = openai.api_type    
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a world class algorithm for extracting information in structured formats from a restaurant order conversation."),
            ("human", """You will receive a transcript of a conversation between a customer and an AI assistant discussing an order made at a restaurant. 
            Your task is to extract only the customer name from the conversation, and nothing else. Input: {input}"""),
            ("human", "Tip: Make sure to answer in the correct format"),
        ]
    )

    # Schema
    schema = {
        "properties": {
            "customer_name": {"type": "string"},
        },
        "required": ["customer_name"],
    }

    # Run chain
    chain = create_extraction_chain(schema=schema, llm=llm, prompt=prompt, verbose=False)
    name = chain.run(chat_str)
    return name[0]['customer_name']

customer_name = extract_customer_name(memory.buffer_as_str)
customer_name

'Shiva'

In [19]:
import pandas as pd

order_df = pd.DataFrame(order_result)
order_df['item'] = order_df['item'].str.strip().str.lower()
order_df

,item,quantity,price,item_notes
0,cafe au lait,2,3.5,"Very hot, less sugar"
1,pain au chocolat,2,3.5,
2,mocha,1,4.5,
3,lemon cake,1,4.0,
4,blueberry smoothie,1,5.5,
5,chicken sandwich,1,5.5,


In [20]:
import pandas as pd  
from fuzzywuzzy import process  

menu_data = pd.read_json('../data/contoso.json') 
  
# Convert the 'Item_Name' column to lower case for standardization  
menu_data['item'] = menu_data['item'].str.lower()  
  
# Create a dictionary to store the item_ids  
item_ids = dict(zip(menu_data['item'], menu_data['id']))  
  
# Extract the item_ids for the item_names in final_order_json_list  
for item in order_result:  
    item_name = item['item'].lower()  
  
    # Use fuzzy matching to find the closest matching item in the menu  
    matched_item, _ = process.extractOne(item_name, item_ids.keys())  
  
    id = item_ids.get(matched_item)  
    item['id'] = id  
  
# Print the updated final_order_json_list  
for item in order_result:  
    print(item) 

{'item': 'Cafe au Lait', 'quantity': 2, 'price': 3.5, 'item_notes': 'Very hot, less sugar', 'id': 'c015'}
{'item': 'Pain au Chocolat', 'quantity': 2, 'price': 3.5, 'item_notes': '', 'id': 'b009'}
{'item': 'Mocha', 'quantity': 1, 'price': 4.5, 'item_notes': '', 'id': 'c005'}
{'item': 'Lemon Cake', 'quantity': 1, 'price': 4.0, 'item_notes': '', 'id': 'b010'}
{'item': 'Blueberry Smoothie', 'quantity': 1, 'price': 5.5, 'item_notes': '', 'id': 'sm002'}
{'item': 'Chicken Sandwich', 'quantity': 1, 'price': 5.5, 'item_notes': '', 'id': 's002'}


In [21]:
def calculate_total(order_result):
    sum = 0
    for item in order_result:
        sum += item['price']*item['quantity']
    return sum

print("Your total is: {0}$".format(calculate_total(order_result)))

Your total is: 33.5$


In [24]:
def print_receipt(order_result, customer_name, tax_percent=0.095):
    print("-----------Contoso Coffee Inc-----------\n")
    for item in order_result:
        print("{0} x {1} = {2}$".format(item['item'], item['quantity'], round(item['price']*item['quantity'], 2)))
        if item['item_notes'] != "":
            print("\t Notes: {0}".format(item['item_notes']))
    
    subtotal = calculate_total(order_result)
    tax = round(tax_percent*subtotal , 2)
    total = round(subtotal + tax, 2)
    print("\nSubtotal: {0}$".format(subtotal))
    print("Tax: {0}$".format(tax))
    print("Total: {0}$".format(total))   
    
    print(f"\nThank you for dining with us, {customer_name.title()}!")
    
print_receipt(order_result, customer_name)

-----------Contoso Coffee Inc-----------

Cafe au Lait x 2 = 7.0$
	 Notes: Very hot, less sugar
Pain au Chocolat x 2 = 7.0$
Mocha x 1 = 4.5$
Lemon Cake x 1 = 4.0$
Blueberry Smoothie x 1 = 5.5$
Chicken Sandwich x 1 = 5.5$

Subtotal: 33.5$
Tax: 3.18$
Total: 36.68$

Thank you for dining with us, Shiva!
